In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
import warnings
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.feature_selection import chi2, SelectKBest, SelectFdr, SelectFwe
warnings.filterwarnings("ignore")
import seaborn as sns

In [31]:
x_data = "/Users/anumehaagrawal/Documents/Course_Work/Emotion-Analysis-in-Multimodal-data/Combined/Lexical + audio - three_features_merged.csv"
y_data = "/Users/anumehaagrawal/Documents/Course_Work/Emotion-Analysis-in-Multimodal-data/Combined/y-output - Sheet1.csv"

In [32]:
x_df = pd.read_csv(x_data)
y_df = pd.read_csv(y_data)

y_df = y_df.astype(int, errors='ignore')
x_df = x_df.drop(x_df.columns[0], axis=1)
y_df = y_df.drop(y_df.columns[0], axis=1)
y_df = y_df.drop(y_df.columns[-1], axis=1)
x_df = x_df.abs()
x_df

,0.0003103610031,58.89807156,196.992,142.1601663,74.30483534,599.939292,0.07841212505,0.002936686887,3.097412401,0.1709953368,...,609,284,3.091496101,1.441682911,8,0.531373,0.520348,0.708033,0,0.1
0,0.000411,60.121939,426.000,115.346030,71.907626,559.165817,0.088106,0.003644,3.098069,0.167862,...,1113.0,368.0,2.612676,0.863850,75.0,0.642117,0.000000,0.651181,0.606895,0.000000
1,0.000134,55.265210,271.992,190.902445,71.030606,593.526919,0.114184,0.002480,3.134633,0.202736,...,741.0,288.0,2.724345,1.058855,13.0,0.606516,0.000000,0.000000,0.620860,0.000000
2,0.000202,57.028214,204.984,186.773508,72.728255,586.069219,0.089705,0.001364,3.139515,0.195331,...,714.0,275.0,3.483199,1.341568,7.0,0.528993,0.521192,0.909287,0.514447,0.000000
3,0.000029,48.634250,294.000,186.221891,72.576028,545.329312,0.112451,0.003005,3.136228,0.227116,...,644.0,260.0,2.190476,0.884354,21.0,0.574884,0.510653,0.709780,0.000000,0.000000
4,0.000238,57.741340,246.000,195.292435,72.860039,511.041246,0.106836,0.002307,3.122065,0.213910,...,612.0,268.0,2.487805,1.089431,5.0,0.599124,0.000000,0.000000,0.660757,0.549712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,0.000070,52.423892,408.000,225.065918,71.466905,597.463942,0.166549,0.001616,3.166655,0.233252,...,513.0,230.0,1.257353,0.563725,19.0,0.588511,0.000000,0.813839,0.576384,0.000000
133,0.000189,56.737974,540.120,225.672808,79.112283,588.757340,0.111539,0.000200,3.146009,0.206337,...,889.0,321.0,1.645931,0.594312,21.0,0.655967,0.000000,0.794747,0.611884,0.000000
134,0.000117,54.647290,626.712,131.393269,72.401717,593.077505,0.137905,0.000200,3.106303,0.204870,...,1222.0,390.0,1.949859,0.622295,65.0,0.611179,0.508575,0.953332,0.684415,0.000000
135,0.000195,56.877320,606.168,142.195172,66.336648,599.357254,0.094506,0.000928,3.111328,0.194756,...,1145.0,357.0,1.888915,0.588946,21.0,0.603404,0.502010,0.000000,0.000000,0.000000


In [33]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr = dataset.corr()
    
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr



In [40]:

# clf = RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')
# efs1 = EFS(clf, 
#            min_features=1,
#            max_features=10,
#            scoring='accuracy',
#            print_progress=True,
#            cv=5)


# efs1 = efs1.fit(X_train, y_train[y_train.columns[0:1]])
# print('Best accuracy score: %.2f' % efs1.best_score_)
    
from sklearn import preprocessing
def multiclass_roc_auc_score(y_test, y_pred, average="weighted"):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [46]:

threshold = 0.5
selected_features = [] 
clf = RandomForestClassifier(random_state=20, max_features=10, n_estimators= 100, max_depth=7, criterion='entropy')
#clf = SVC(decision_function_shape='ovo')
#clf = linear_model.MultiTaskLasso(alpha=0.4,random_state=42)
for i in range(len(y_df.columns)):
    
    selector = SelectFwe(alpha =40.0)
    X_new = selector.fit_transform(x_df,  y_df[y_df.columns[i:i+1]])
    X = pd.DataFrame(X_new)
    X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=20)
    corr_features = correlation(X_train, 0.8)
    
    X_train.drop(labels=corr_features, axis=1, inplace=True)
    X_test.drop(labels=corr_features, axis=1, inplace=True)
    clf.fit(X_train, y_train[y_train.columns[i:i+1]])
    y_pred = clf.predict(X_test)
    y_predict = pd.DataFrame(y_pred)
    #macro_roc_auc_ovo = multiclass_roc_auc_score(y_test[y_test.columns[i:i+1]],y_predict)
    #print(macro_roc_auc_ovo)
    print(accuracy_score(y_test[y_test.columns[i:i+1]],y_predict))




0.6071428571428571
0.39285714285714285
0.42857142857142855
0.75
0.32142857142857145
0.32142857142857145
0.4642857142857143
0.7857142857142857
0.4642857142857143
0.6071428571428571
0.6785714285714286
0.5
0.42857142857142855
0.7142857142857143
0.75
0.6785714285714286
0.6071428571428571
0.4642857142857143


In [16]:
from sklearn.neural_network import MLPClassifier
threshold = 0.6
from sklearn.preprocessing import StandardScaler  # doctest: +SKIP

clf = RandomForestClassifier(random_state=20, max_features=10, n_estimators= 100, max_depth=7, criterion='entropy')
for i in range(len(y_df.columns)):
    
    selector = SelectFdr(chi2, alpha =11.0)
    X_new = selector.fit_transform(x_df,  y_df[y_df.columns[i:i+1]])
    X = pd.DataFrame(X_new)
    X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=42)
    scaler = StandardScaler()  # doctest: +SKIP
    scaler.fit(X_train)  # doctest: +SKIP
    X_train = scaler.transform(X_train)  # doctest: +SKIP
    X_test = scaler.transform(X_test) 
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    corr_features = correlation(X_train, 0.8)
    X_train.drop(labels=corr_features, axis=1, inplace=True)
    X_test.drop(labels=corr_features, axis=1, inplace=True)
    clf.fit(X_train, y_train[y_train.columns[i:i+1]])
    y_pred = clf.predict(X_test)
    y_predict = pd.DataFrame(y_pred)
    #macro_roc_auc_ovo = multiclass_roc_auc_score(y_test[y_test.columns[i:i+1]],y_predict)
    #print(macro_roc_auc_ovo)
    print(accuracy_score(y_test[y_test.columns[i:i+1]],y_predict))

0.6071428571428571
0.6428571428571429
0.5714285714285714
0.6071428571428571
0.42857142857142855
0.5357142857142857
0.32142857142857145
0.9285714285714286
0.2857142857142857
0.42857142857142855
0.8214285714285714
0.5
0.39285714285714285
0.7857142857142857
0.8214285714285714
0.6071428571428571
0.6428571428571429
0.35714285714285715


In [67]:
from sklearn.neural_network import MLPClassifier
threshold = 0.6
from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
from sklearn.model_selection import KFold 
kf = KFold(4,True,4) 
clf = RandomForestClassifier( random_state= 10,max_features=10, n_estimators= 100, max_depth=9, criterion='entropy')
for i in range(len(y_df.columns)):
    selector = SelectFdr(chi2, alpha =9.0)
    X_new = selector.fit_transform(x_df,  y_df[y_df.columns[i:i+1]])
    X = pd.DataFrame(X_new)
    summ =0
    count =0
    for train_index, val_index in kf.split(X):
        count = count+1
        X_train, X_test = X.loc[train_index], X.loc[val_index]
        y_train, y_test = y_df.loc[train_index], y_df.loc[val_index]
        
        scaler = StandardScaler()  # doctest: +SKIP
        scaler.fit(X_train)  # doctest: +SKIP
        X_train = scaler.transform(X_train)  # doctest: +SKIP
        X_test = scaler.transform(X_test) 
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        corr_features = correlation(X_train, 0.8)
        X_train.drop(labels=corr_features, axis=1, inplace=True)
        X_test.drop(labels=corr_features, axis=1, inplace=True)
        clf.fit(X_train, y_train[y_train.columns[i:i+1]])
        y_pred = clf.predict(X_test)
        y_predict = pd.DataFrame(y_pred)
        #macro_roc_auc_ovo = multiclass_roc_auc_score(y_test[y_test.columns[i:i+1]],y_predict)
        #print(macro_roc_auc_ovo)
        summ = summ + accuracy_score(y_test[y_test.columns[i:i+1]],y_predict)
    print(summ/count)


0.48865546218487393
0.36470588235294116
0.5033613445378151
0.5903361344537815
0.3871848739495798
0.5327731092436975
0.38802521008403357
0.8172268907563025
0.32815126050420174
0.45336134453781507
0.6716386554621848
0.3867647058823529
0.5109243697478991
0.6487394957983192
0.6415966386554621
0.6852941176470588
0.51890756302521
0.3058823529411765


In [72]:
from sklearn.model_selection import GridSearchCV
threshold = 0.6
from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100],
    'max_features': [2, 8, 10, 7],
    'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
clf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = 6, verbose = 2)
#clf = RandomForestClassifier(random_state=20, max_features=10, n_estimators= 100, max_depth=7, criterion='entropy')
for i in range(len(y_df.columns)):
    
    selector = SelectFdr(chi2, alpha =11.0)
    X_new = selector.fit_transform(x_df,  y_df[y_df.columns[i:i+1]])
    X = pd.DataFrame(X_new)
    X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=42)
    scaler = StandardScaler()  # doctest: +SKIP
    scaler.fit(X_train)  # doctest: +SKIP
    X_train = scaler.transform(X_train)  # doctest: +SKIP
    X_test = scaler.transform(X_test) 
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    corr_features = correlation(X_train, 0.8)
    X_train.drop(labels=corr_features, axis=1, inplace=True)
    X_test.drop(labels=corr_features, axis=1, inplace=True)
    clf.fit(X_train, y_train[y_train.columns[i:i+1]])
    y_pred = clf.predict(X_test)
    y_predict = pd.DataFrame(y_pred)
    #macro_roc_auc_ovo = multiclass_roc_auc_score(y_test[y_test.columns[i:i+1]],y_predict)
    #print(macro_roc_auc_ovo)
    print(accuracy_score(y_test[y_test.columns[i:i+1]],y_predict))


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    8.0s
[Parallel(n_jobs=6)]: Done  97 out of 108 | elapsed:    8.5s remaining:    1.0s
[Parallel(n_jobs=6)]: Done 108 out of 108 | elapsed:    8.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


0.6071428571428571
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=6)]: Done 108 out of 108 | elapsed:    0.8s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


0.42857142857142855
Fitting 3 folds for each of 36 candidates, totalling 108 fits


...

[Parallel(n_jobs=6)]: Done 108 out of 108 | elapsed:    1.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


0.6071428571428571
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=6)]: Done 108 out of 108 | elapsed:    1.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


0.35714285714285715
Fitting 3 folds for each of 36 candidates, totalling 108 fits
0.6071428571428571


[Parallel(n_jobs=6)]: Done 108 out of 108 | elapsed:    1.7s finished


In [ ]:
0.6071428571428571
0.42857142857142855
0.4642857142857143
0.6071428571428571
0.39285714285714285
0.5714285714285714
0.4642857142857143
0.9642857142857143
0.32142857142857145
0.39285714285714285
0.8214285714285714
0.42857142857142855
0.42857142857142855
0.6428571428571429
0.7142857142857143
0.6071428571428571
0.35714285714285715
0.6071428571428571